<a href="https://colab.research.google.com/github/regraded0101/pytorch/blob/main/sonar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
sonar_df = pd.read_csv('sonar_csv.csv')
X_np = sonar_df.drop('Class', axis=1).values
X_np = X_np.astype(np.float32)

# Mine = 0, Rock = 1, no NA values
target_np = np.where(sonar_df['Class'] == 'Mine', 0, 1)
target_np = target_np.astype(np.float32)

FileNotFoundError: ignored

In [ ]:
from sklearn.model_selection import train_test_split

X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_np, target_np,
    random_state=42, test_size=0.3
)

In [ ]:
import torch

In [ ]:
X_train = torch.tensor(X_train_np)
y_train = torch.tensor(y_train_np)
X_test = torch.tensor(X_test_np)
y_test = torch.tensor(y_test_np)


In [ ]:
from torch import nn

In [ ]:
class Deep(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(60,60)
    self.act1 = nn.ReLU()
    self.layer2 = nn.Linear(60,180)
    self.act2 = nn.ReLU()
    self.layer3 = nn.Linear(180,60)
    self.act3 = nn.ReLU()
    self.output = nn.Linear(60,1)
    self.sig = nn.Sigmoid()

  def forward(self, x):
    x = self.act1(self.layer1(x))
    x = self.act2(self.layer2(x))
    x = self.act3(self.layer3(x))
    x = self.sig(self.output(x))
    return x



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_deep = Deep()
model_deep = model_deep.to(device)

X_train = X_train.to(device)
y_train = y_train.to(device).unsqueeze(-1)
X_test = X_test.to(device)
y_test = y_test.to(device).unsqueeze(-1)
print(f"Running on :{device}")


Running on :cuda


In [ ]:
loss_func = nn.BCELoss()
optim = torch.optim.Adam(
    params=model_deep.parameters(),
    lr=0.0001
)

In [ ]:
from tqdm import tqdm
n_epochs = 50
batch_size = 10

batch_id = torch.arange(start=0, end=len(X_train), step=batch_size)

for epoch in range(n_epochs):
  for batch in batch_id:
    model_deep.train()

    X_batch = X_train[batch:batch + batch_size]
    y_batch = y_train[batch:batch + batch_size]

    y_pred = model_deep(X_batch)
    loss = loss_func(y_pred, y_batch)

    optim.zero_grad()
    loss.backward()

    # update weights
    optim.step()

  if epoch % 10 == 0:
    print(f"Epoch: {epoch}, Loss: {loss.item()}")
    # calculate the test accuracy
    model_deep.eval()
    y_val = model_deep(X_test)
    acc = (y_val.round() == y_test).float().mean()
    print(f"Test Accuracy: {acc}")

    # calculate the training accuracy
    y_train_val = model_deep(X_train)
    acc_train = (y_train_val.round() == y_train).float().mean()
    print(f"Train Accuracy: {acc_train}\n")




Epoch: 0, Loss: 0.7073397040367126
Test Accuracy: 0.444444477558136
Train Accuracy: 0.475862056016922

Epoch: 10, Loss: 0.6804347038269043
Test Accuracy: 0.6666666865348816
Train Accuracy: 0.5448275804519653

Epoch: 20, Loss: 0.659959614276886
Test Accuracy: 0.7619048357009888
Train Accuracy: 0.6896551847457886

Epoch: 30, Loss: 0.6102162599563599
Test Accuracy: 0.8253968954086304
Train Accuracy: 0.7862069010734558

Epoch: 40, Loss: 0.5398090481758118
Test Accuracy: 0.8412699103355408
Train Accuracy: 0.834482729434967



In [ ]:
10 % 10 == 0

True

In [ ]:
X_batch.dtype

torch.float64